In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from control.matlab import *
import scipy as sci 

In [2]:
# First we load the system to perform an impulse response. We are nos suposed to know the system
testSys_mat = sci.io.loadmat('DATA/testSys_ABCD.mat')
A = testSys_mat['A']
B = testSys_mat['B']
C = testSys_mat['C']
D = testSys_mat['D']

sysFull = ss(A,B,C,D,1)

In [3]:
q = 2 #Number of inputs 
p = 2 #Number of outputs 
n = 100 #state dimension 
r = 10 #reduced model order

In [11]:
tspan = np.arange(0,r*5+2,1)
yFull = np.array([])
# Now we do an impulse response for each input variable 
for qi in range(q):
    yaux, t = impulse(sysFull,T=tspan,input=qi)
    if qi==0:
        yFull=yaux
    else:
        yFull = np.concatenate((yFull,yaux),axis=2)

In [13]:
YY = np.transpose(yFull,axes=(1,2,0)) #size pxqxm

In [ ]:
# Now we define the ERA and OKID functions 
def ERA(YY,m,n,nin,nout,r): 
    """ 
    This function computes the Eigen-System-Realization of impulse response data. 
    Inputs: 
        - Y: Impulse response measurements in pxqxm format (n_outputs,n_inputs,temporal data)
        - r: Order of reduction
        
    Outputs: 
        - model: A, B, C and D
        - HSVs: Henkel Singular Values (Sigma)
    """
    # First we assign the Dr matrix with the available data 
    Dr = np.zeros((nout,nin))
    Y = np.zeros((nout,nin,YY.shape[2]-1))
    
    for i in range(nout):
        for j in range(nin):
            Dr[i,j]=YY[i,j,0]
            Y[i,j,:]=YY[i,j,1:]

    assert len(Y[:,1,1]) == nout
    assert len(Y[1,:,1]) == nin
    assert len(Y[1,1,:]) >= m+n   
    
    H = np.zeros((nout*m,nin*n))
    H2 = np.zeros((nout*m,nin*n))
    
    for i in range(m):
        for j in range(n):
            for Q in range(nout):
                for P in range(nin):
                    H[nout*i+Q,nin*j+P] = Y[Q,P,i+j]
                    H2[nout*i+Q,nin*j+P] = Y[Q,P,i+j+1]
    

    U,S,VT = np.linalg.svd(H,full_matrices=0)
    V = VT.T 
    Sigma = np.diag(S[:r]) 
    Ur = U[:,:r]
    Vr = V[:,:r]
    Ar = fractional_matrix_power(Sigma,-0.5) @ Ur.T @ H2 @ Vr @ fractional_matrix_power(Sigma,-0.5)
    Br = fractional_matrix_power(Sigma,-0.5) @ Ur.T @ H[:,:nin]
    Cr = H[:nout,:] @ Vr @ fractional_matrix_power(Sigma,-0.5)
    HSVs = S
    
    return Ar,Br,Cr,Dr,HSVs


In [15]:
len(YY[0,0,:])

52